设置工作目录

In [1]:
train_folder = 'train'
test_folder = 'test'
my_train_folder='my_train'
my_validation_folder = 'my_validation'

建立训练集、验证集目录

In [2]:
num_classes=10
#classes=["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
import os
if os.path.exists("my_validation") is False:
    os.mkdir("my_validation")    
    for j in range(num_classes):
        path = os.path.join(my_validation_folder, 'c' + str(j))
        os.mkdir(path)
if os.path.exists("my_train") is False:
    os.mkdir("my_train")    
    for j in range(num_classes):
        path = os.path.join(my_train_folder, 'c' + str(j))
        os.mkdir(path)

统计目录下各类图片数量

In [3]:
import glob
import numpy as np 
def img_number(path,num_classes=10):
    train_num_files=[]
    for j in range(num_classes):
        train_path = os.path.join(path, 'c' + str(j), '*.jpg')
        train_files = glob.glob(train_path)
        train_num_files.append(np.size(train_files))
    train= [('class ' + str(i)+ ' : '+ str(train_num_files[i])) for i in range(10)] 
    print(train,sum(train_num_files))
    return train_num_files,sum(train_num_files)  

img_number(train_folder)    
img_number(my_train_folder)
img_number(my_validation_folder)


['class 0 : 2489', 'class 1 : 2267', 'class 2 : 2317', 'class 3 : 2346', 'class 4 : 2326', 'class 5 : 2312', 'class 6 : 2325', 'class 7 : 2002', 'class 8 : 1911', 'class 9 : 2129'] 22424
['class 0 : 2357', 'class 1 : 2132', 'class 2 : 2173', 'class 3 : 2211', 'class 4 : 2194', 'class 5 : 2179', 'class 6 : 2187', 'class 7 : 1870', 'class 8 : 1781', 'class 9 : 1995'] 21079
['class 0 : 132', 'class 1 : 135', 'class 2 : 144', 'class 3 : 135', 'class 4 : 132', 'class 5 : 133', 'class 6 : 138', 'class 7 : 132', 'class 8 : 130', 'class 9 : 134'] 1345


([132, 135, 144, 135, 132, 133, 138, 132, 130, 134], 1345)

读取driver_imgs_list文件

In [4]:
def get_driver_data():
    dr_dict = {}
    dr_set = set()
    path = os.path.join('driver_imgs_list.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')   
        dr_dict[arr[2]] = arr[0]
        dr_set.add(arr[0])
    f.close()
    return dr_dict,dr_set

建立训练集、验证集图片(随机选取两位司机)

In [6]:
driver_img,driver_id=get_driver_data()
import random
driver_id_1=(random.choice(list(driver_id)))
driver_id_2=(random.choice(list(driver_id)))
if driver_id_1==driver_id_2:
    driver_id_2=(random.choice(list(driver_id)))
print(driver_id_1,driver_id_2)    
def creative_train_val_dataset(numclasses=10):
    for j in range(num_classes):
        train_path = os.path.join(train_folder, 'c' + str(j), '*.jpg')
        print(train_path)
        train_files = glob.glob(train_path)
        for eachfile in train_files:
            v_path=os.path.join(my_validation_folder, 'c' + str(j))
            t_path=os.path.join(my_train_folder, 'c' + str(j))
            if driver_img[eachfile.split('\\')[-1].split('.')[0]+'.jpg'] in (driver_id_1,driver_id_2):
                os.system ("copy %s %s" % (eachfile, v_path))
            else:
                os.system ("copy %s %s" % (eachfile, t_path))
creative_train_val_dataset(numclasses=10)

Read drivers data
p041 p052
train\c0\*.jpg
train\c1\*.jpg
train\c2\*.jpg
train\c3\*.jpg
train\c4\*.jpg
train\c5\*.jpg
train\c6\*.jpg
train\c7\*.jpg
train\c8\*.jpg
train\c9\*.jpg


In [5]:
img_number(train_folder)    
img_number(my_train_folder)
img_number(my_validation_folder)

['class 0 : 2489', 'class 1 : 2267', 'class 2 : 2317', 'class 3 : 2346', 'class 4 : 2326', 'class 5 : 2312', 'class 6 : 2325', 'class 7 : 2002', 'class 8 : 1911', 'class 9 : 2129'] 22424
['class 0 : 2357', 'class 1 : 2132', 'class 2 : 2173', 'class 3 : 2211', 'class 4 : 2194', 'class 5 : 2179', 'class 6 : 2187', 'class 7 : 1870', 'class 8 : 1781', 'class 9 : 1995'] 21079
['class 0 : 132', 'class 1 : 135', 'class 2 : 144', 'class 3 : 135', 'class 4 : 132', 'class 5 : 133', 'class 6 : 138', 'class 7 : 132', 'class 8 : 130', 'class 9 : 134'] 1345


([132, 135, 144, 135, 132, 133, 138, 132, 130, 134], 1345)

图片预处理及增强

In [6]:
from keras.applications import vgg16
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.applications import inception_resnet_v2
from keras.preprocessing.image import ImageDataGenerator
img_height, img_width=(299,299)

# prepare data augmentation configuration
vgg_train_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
vgg_validation_datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

resnet50_train_datagen = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)
resnet50_validation_datagen = ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

inceptionv3_train_datagen = ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input)
inceptionv3_validation_datagen = ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input)

inceptionresnetv2_train_datagen = ImageDataGenerator(preprocessing_function=inception_resnet_v2.preprocess_input)
inceptionresnetv2_validation_datagen = ImageDataGenerator(preprocessing_function=inception_resnet_v2.preprocess_input)

print("vgg train/validation loading")
print("----------------------------")
print("")
vgg_train_generator = vgg_train_datagen.flow_from_directory(
        my_train_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')
vgg_validation_generator = vgg_validation_datagen.flow_from_directory(
        my_validation_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

print("resent50 train/validation loading")
print("---------------------------------")
print("")
resnet50_train_generator = resnet50_train_datagen.flow_from_directory(
        my_train_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')
resnet50_validation_generator = resnet50_validation_datagen.flow_from_directory(
        my_validation_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

print("inceptionv3 train/validation loading")
print("------------------------------------")
print("")
inceptionv3_train_generator = inceptionv3_train_datagen.flow_from_directory(
        my_train_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')
inceptionv3_validation_generator = inceptionv3_validation_datagen.flow_from_directory(
        my_validation_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

print("inceptionresnetv2 train/validation loading")
print("------------------------------------------")
print("")
inceptionresnetv2_train_generator = inceptionresnetv2_train_datagen.flow_from_directory(
        my_train_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')
inceptionresnetv2_validation_generator = inceptionresnetv2_validation_datagen.flow_from_directory(
        my_validation_folder,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

C:\Users\Q\Anaconda2\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


vgg train/validation loading
----------------------------

Found 21079 images belonging to 10 classes.
Found 1345 images belonging to 10 classes.
resent50 train/validation loading
---------------------------------

Found 21079 images belonging to 10 classes.
Found 1345 images belonging to 10 classes.
inceptionv3 train/validation loading
------------------------------------

Found 21079 images belonging to 10 classes.
Found 1345 images belonging to 10 classes.
inceptionresnetv2 train/validation loading
------------------------------------------

Found 21079 images belonging to 10 classes.
Found 1345 images belonging to 10 classes.


神经网络初始化

In [7]:
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras import backend as K
from keras import optimizers
from keras import regularizers

vgg_base_model = VGG16(weights='imagenet', include_top=False)
resnet50_base_model = ResNet50(weights='imagenet', include_top=False)
inceptionv3_base_model = InceptionV3(weights='imagenet', include_top=False)
inceptionresnetv2_base_model = InceptionResNetV2(weights='imagenet', include_top=False)

In [8]:
#for i, layer in enumerate(vgg_base_model.layers):
#   print(i, layer.name)
#print("---------------------------------------")
#for i, layer in enumerate(resnet50_base_model.layers):
#   print(i, layer.name)
#print("---------------------------------------")
#for i, layer in enumerate(inceptionv3_base_model.layers):
#   print(i, layer.name)
#print("---------------------------------------")
#for i, layer in enumerate(inceptionresnetv2_base_model.layers):
#   print(i, layer.name)

建立VGG16神经网络

In [9]:
# add a global spatial average pooling layer
x_vgg = vgg_base_model.output
x_vgg = GlobalAveragePooling2D()(x_vgg)

# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)

# let's add a Dropout layer
x_vgg = Dropout(0.2)(x_vgg)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x_vgg)

# this is the model we will train
vgg_model = Model(inputs=vgg_base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in vgg_base_model.layers:
#    layer.trainable = False

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:    
for layer in vgg_base_model.layers[:15]:
   layer.trainable = False
for layer in vgg_base_model.layers[15:]:
   layer.trainable = True    
sgd = optimizers.SGD(lr=1e-4, momentum=0.9)
# compile the model (should be done *after* setting layers to non-trainable)
vgg_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [10]:
# add a global spatial average pooling layer
x_resnet = resnet50_base_model.output
x_resnet = GlobalAveragePooling2D()(x_resnet)

# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)

# let's add a Dropout layer
x_resnet = Dropout(0.2)(x_resnet)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x_resnet)

# this is the model we will train
resnet_model = Model(inputs=resnet50_base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in vgg_base_model.layers:
#    layer.trainable = False

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:    
for layer in resnet50_base_model.layers[:151]:
   layer.trainable = False
for layer in resnet50_base_model.layers[151:]:
   layer.trainable = True    
sgd = optimizers.SGD(lr=1e-3, momentum=0.9)
# compile the model (should be done *after* setting layers to non-trainable)
resnet_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [11]:
# add a global spatial average pooling layer
x_inceptionv3 = inceptionv3_base_model.output
x_inceptionv3 = GlobalAveragePooling2D()(x_inceptionv3)

# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)

# let's add a Dropout layer
x_inceptionv3 = Dropout(0.2)(x_inceptionv3)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x_inceptionv3)

# this is the model we will train
inceptionv3_model = Model(inputs=inceptionv3_base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in vgg_base_model.layers:
#    layer.trainable = False

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:    
for layer in inceptionv3_base_model.layers[:249]:
   layer.trainable = False
for layer in inceptionv3_base_model.layers[249:]:
   layer.trainable = True    
sgd = optimizers.SGD(lr=1e-3, momentum=0.9)
# compile the model (should be done *after* setting layers to non-trainable)
inceptionv3_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [12]:
# add a global spatial average pooling layer
x_inceptionresnetv2 = inceptionresnetv2_base_model.output
x_inceptionresnetv2 = GlobalAveragePooling2D()(x_inceptionresnetv2)

# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)

# let's add a Dropout layer
x_inceptionresnetv2 = Dropout(0.2)(x_inceptionresnetv2)

# and a logistic layer -- let's say we have 10 classes
predictions = Dense(10, activation='softmax')(x_inceptionresnetv2)

# this is the model we will train
inceptionresnetv2_model = Model(inputs=inceptionresnetv2_base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in vgg_base_model.layers:
#    layer.trainable = False

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:    
for layer in inceptionresnetv2_base_model.layers[:698]:
   layer.trainable = False
for layer in inceptionresnetv2_base_model.layers[698:]:
   layer.trainable = True    
sgd = optimizers.SGD(lr=1e-3, momentum=0.9)
# compile the model (should be done *after* setting layers to non-trainable)
inceptionresnetv2_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [15]:
# fine-tune the model
vgg_model.fit_generator(
        vgg_train_generator,
        steps_per_epoch=672,
        epochs=20,
        validation_data=vgg_validation_generator,
        validation_steps=32)
# fine-tune the model
resnet_model.fit_generator(
        resnet50_train_generator,
        steps_per_epoch=672,
        epochs=20,
        validation_data=resnet50_validation_generator,
        validation_steps=32)
# fine-tune the model
inceptionv3_model.fit_generator(
        inceptionv3_train_generator,
        steps_per_epoch=672,
        epochs=20,
        validation_data=inceptionv3_validation_generator,
        validation_steps=32)
inceptionresnetv2_model.fit_generator(
        inceptionresnetv2_train_generator,
        steps_per_epoch=672,
        epochs=20,
        validation_data=inceptionresnetv2_validation_generator,
        validation_steps=32)

Epoch 1/20
672/672 [==============================] - 355s 528ms/step - loss: 0.0908 - categorical_accuracy: 0.9760 - val_loss: 0.1358 - val_categorical_accuracy: 0.9707
Epoch 2/20
672/672 [==============================] - 354s 527ms/step - loss: 0.0582 - categorical_accuracy: 0.9859 - val_loss: 0.3907 - val_categorical_accuracy: 0.8781
Epoch 3/20
672/672 [==============================] - 352s 524ms/step - loss: 0.0411 - categorical_accuracy: 0.9900 - val_loss: 0.3321 - val_categorical_accuracy: 0.9013
Epoch 4/20
672/672 [==============================] - 354s 527ms/step - loss: 0.0308 - categorical_accuracy: 0.9931 - val_loss: 0.3481 - val_categorical_accuracy: 0.8936
Epoch 5/20
672/672 [==============================] - 353s 525ms/step - loss: 0.0239 - categorical_accuracy: 0.9950 - val_loss: 0.0941 - val_categorical_accuracy: 0.9829
Epoch 6/20
672/672 [==============================] - 352s 524ms/step - loss: 0.0204 - categorical_accuracy: 0.9952 - val_loss: 0.2845 - val_categoric

Epoch 9/20
672/672 [==============================] - 487s 724ms/step - loss: 0.0029 - categorical_accuracy: 0.9999 - val_loss: 2.6848 - val_categorical_accuracy: 0.2669
Epoch 10/20
672/672 [==============================] - 485s 722ms/step - loss: 0.0027 - categorical_accuracy: 0.9999 - val_loss: 2.8136 - val_categorical_accuracy: 0.2538
Epoch 11/20
672/672 [==============================] - 487s 724ms/step - loss: 0.0022 - categorical_accuracy: 1.0000 - val_loss: 3.1174 - val_categorical_accuracy: 0.2518
Epoch 12/20
672/672 [==============================] - 482s 717ms/step - loss: 0.0021 - categorical_accuracy: 1.0000 - val_loss: 2.8878 - val_categorical_accuracy: 0.2080
Epoch 13/20
672/672 [==============================] - 482s 717ms/step - loss: 0.0019 - categorical_accuracy: 1.0000 - val_loss: 2.8505 - val_categorical_accuracy: 0.2608
Epoch 14/20
672/672 [==============================] - 482s 718ms/step - loss: 0.0016 - categorical_accuracy: 1.0000 - val_loss: 2.9361 - val_cate